In [4]:
#importing libraries

import requests
import pandas as pd
from pandas import json_normalize

In [5]:
# importing bikes df from before

bikes_df = pd.read_csv('bikes.csv')

# Foursquare

In [7]:
# bike data was converted into a dictionary
bikes_coordinates_dict = {}

for index, row in bikes_df.iterrows():
    lat, lon = (row['latitude'], row['longitude'])
    bike_stop = {index + 1:{'index': index + 1, 'coordinates': (lat,lon)}}
    bikes_coordinates_dict.update(bike_stop)

bikes_coordinates_dict

{1: {'index': 1, 'coordinates': (41.4299999, 2.190246)},
 2: {'index': 2, 'coordinates': (41.4433647, 2.1906302)},
 3: {'index': 3, 'coordinates': (41.3775319, 2.1707321)},
 4: {'index': 4, 'coordinates': (41.3751161, 2.1523091)},
 5: {'index': 5, 'coordinates': (41.4090202, 2.1954152)},
 6: {'index': 6, 'coordinates': (41.4180789, 2.1763994)},
 7: {'index': 7, 'coordinates': (41.3939378, 2.1380304)},
 8: {'index': 8, 'coordinates': (41.4092333, 2.1557747)},
 9: {'index': 9, 'coordinates': (41.4183999, 2.2058133)},
 10: {'index': 10, 'coordinates': (41.3545425, 2.1433038)},
 11: {'index': 11, 'coordinates': (41.426925, 2.1786224)},
 12: {'index': 12, 'coordinates': (41.4216738, 2.1799878)},
 13: {'index': 13, 'coordinates': (41.4283127, 2.1629889)},
 14: {'index': 14, 'coordinates': (41.3934324, 2.115107)},
 15: {'index': 15, 'coordinates': (41.3875297, 2.1237141)},
 16: {'index': 16, 'coordinates': (41.4280668, 2.1443777)},
 17: {'index': 17, 'coordinates': (41.4354601, 2.2001569)},
 

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [9]:
# 200 meter was chosen due to the closeness of the bike stops. 
# Many bikes stops were 300 - 500 meters away from a neighbouring stop.

foursquare_responses = []

for bike_stop in bikes_coordinates_dict.values():
    index = bike_stop['index']
    lat, lon = bike_stop['coordinates']
    url = f"https://api.foursquare.com/v3/places/search?query=museum&ll={lat},{lon}&radius=200"

    headers = {
        "accept": "application/json",
        "Authorization": open('C:/users/ellis/Lighthouse Labs/Lighthouse Tests +Projects/Statistical Modelling with Python/FoursqaureApiKey.txt').readlines()[0].strip()
    }

    foursquare_response = requests.get(url, headers=headers)
    foursquare_responses.append({'index': index, "data": foursquare_response.json()})
    
foursquare_responses

[{'index': 1,
  'data': {'results': [],
   'context': {'geo_bounds': {'circle': {'center': {'latitude': 41.4299999,
       'longitude': 2.190246},
      'radius': 200}}}}},
 {'index': 2,
  'data': {'results': [],
   'context': {'geo_bounds': {'circle': {'center': {'latitude': 41.4433647,
       'longitude': 2.1906302},
      'radius': 200}}}}},
 {'index': 3,
  'data': {'results': [],
   'context': {'geo_bounds': {'circle': {'center': {'latitude': 41.3775319,
       'longitude': 2.1707321},
      'radius': 200}}}}},
 {'index': 4,
  'data': {'results': [{'fsq_id': '5aa5a85046f84dfd0a4c60bb',
     'categories': [{'id': 10028,
       'name': 'Art Museum',
       'short_name': 'Art Museum',
       'plural_name': 'Art Museums',
       'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
        'suffix': '.png'}},
      {'id': 18053,
       'name': 'Track',
       'short_name': 'Track',
       'plural_name': 'Tracks',
       'icon': {'prefix': 'https://

In [ ]:
# and saved to .json file (available in file: data > foursquare_output.json

import json
with open('foursquare_output.json', 'w') as json_file:
    json.dump(foursquare_responses, json_file)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame.

In [10]:
foursquare_df = pd.DataFrame()

#iterate through to create dataframe
for response in foursquare_responses:
    data = response['data']['results']
    
    #keep bike stations that do not have any nearby museums
    if not data:
        row = {'Bike Stop #': response['index'], 'fsq_id': None, 'name': None, 'categories': None, 'distance': None}
        row_df = pd.DataFrame([row])
        foursquare_df = pd.concat([foursquare_df, row_df], ignore_index=True)
    else:
        normalized_data = pd.json_normalize(data)
    
        bike_stop_index = response['index']
        normalized_data['Bike Stop #'] = bike_stop_index
    
        foursquare_df = pd.concat([foursquare_df, normalized_data], ignore_index=True)

foursquare_df

,Bike Stop #,fsq_id,name,categories,distance,chains,closed_bucket,link,geocodes.drop_off.latitude,geocodes.drop_off.longitude,...,location.locality,location.postcode,location.region,timezone,location.cross_street,location.address_extended,related_places.parent.fsq_id,related_places.parent.categories,related_places.parent.name,related_places.children
0,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5aa5a85046f84dfd0a4c60bb,Montjuic (Barcelona),"[{'id': 10028, 'name': 'Art Museum', 'short_na...",181,[],LikelyOpen,/v3/places/5aa5a85046f84dfd0a4c60bb,41.376777,2.151849,...,Barcelona,08015,Barcelona,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,502,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629,503,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
630,504,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,505,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# parsing and cleaning columns. Mostly only require names and distance from a bike station

foursquare_df_cleaning = foursquare_df.drop(columns=['fsq_id'] + list(foursquare_df.columns[5:]))
foursquare_df_cleaning['venue_type'] = foursquare_df_cleaning['categories'].apply(lambda x: [y['name'] for y in x][0] if x is not None else [])
final_foursquare_df = foursquare_df_cleaning.drop(columns=['categories'])

#resulting dataframe example can be found in file: images > foursquare_df_examples.png
final_foursquare_df

,Bike Stop #,name,distance,venue_type
0,1,None,None,[]
1,2,None,None,[]
2,3,None,None,[]
3,4,Montjuic (Barcelona),181,Art Museum
4,5,None,None,[]
...,...,...,...,...
628,502,None,None,[]
629,503,None,None,[]
630,504,None,None,[]
631,505,None,None,[]


In [13]:
# saved to csv file

final_foursquare_df.to_csv('foursquare.csv')

# Yelp

In [14]:
# as seen above

import requests
import pandas as pd
from pandas import json_normalize

In [15]:
bikes_coordinates_dict = {}

for index, row in bikes_df.iterrows():
    lat, lon = (row['latitude'], row['longitude'])
    bike_stop = {index + 1:{'index': index + 1, 'coordinates': (lat,lon)}}
    bikes_coordinates_dict.update(bike_stop)

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [16]:
# 200 meter was chosen due to the closeness of the bike stops as before 

import requests

yelp_responses = []

for bike_stop in bikes_coordinates_dict.values():
    index = bike_stop['index']
    lat, lon = bike_stop['coordinates']

    url = f"https://api.yelp.com/v3/businesses/search"
    
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer ' + open('C:/users/ellis/Lighthouse Labs/Lighthouse Tests +Projects/Statistical Modelling with Python/YelpApiKey.txt').readlines()[0].strip()
    }

    params = {
        'latitude': lat,
        'longitude': lon,
        'term': 'museum',
        'radius': 200
    }

    yelp_response = requests.get(url, headers=headers, params=params)
    
    yelp_responses.append({'index': index, "data": yelp_response.json()})
    
yelp_responses

[{'index': 1,
  'data': {'businesses': [],
   'total': 0,
   'region': {'center': {'longitude': 2.190246, 'latitude': 41.4299999}}}},
 {'index': 2,
  'data': {'businesses': [],
   'total': 0,
   'region': {'center': {'longitude': 2.1906302, 'latitude': 41.4433647}}}},
 {'index': 3,
  'data': {'businesses': [{'id': 'aPXo4l_JGB_j9v6e6eECvQ',
     'alias': 'palais-guëll-barcelona',
     'name': 'Palais Guëll',
     'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/F1P5TjtDLk19phxlsZKs2g/o.jpg',
     'is_closed': False,
     'url': 'https://www.yelp.com/biz/palais-gu%C3%ABll-barcelona?adjust_creative=2GV4Yz-WJ9nPioOtR6zaeQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2GV4Yz-WJ9nPioOtR6zaeQ',
     'review_count': 2,
     'categories': [{'alias': 'castles', 'title': 'Castles'},
      {'alias': 'museums', 'title': 'Museums'}],
     'rating': 4.5,
     'coordinates': {'latitude': 41.3772634, 'longitude': 2.1723679},
     'transactions': [],
     'location': {'addre

In [ ]:
# saved to a .json file for potential future use. Found in file: data > yelp_output.json

with open('yelp_output.json', 'w') as json_file:
     json.dump(yelp_responses, json_file)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [17]:
yelp_df = pd.DataFrame()

# convert to dataframe first

for response in yelp_responses:
    data = response['data']
    
    #keep bike stations that do not have results and set to none
    
    if 'businesses' not in data or not data['businesses']:
        # Handle the case when there are no businesses or 'businesses' is not present
        row = {'Bike Stop #': response['index'], 'fsq_id': None, 'name': None, 'categories': None, 'distance': None}
        row_df = pd.DataFrame([row])
        yelp_df = pd.concat([yelp_df, row_df], ignore_index=True)
    else:
        normalized_data = pd.json_normalize(data['businesses'])
    
        bike_stop_index = response['index']
        normalized_data['Bike Stop #'] = bike_stop_index
    
        yelp_df = pd.concat([yelp_df, normalized_data], ignore_index=True)
        
yelp_df

,Bike Stop #,fsq_id,name,categories,distance,id,alias,image_url,is_closed,url,...,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,price
0,1,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,Palais Guëll,"[{'alias': 'castles', 'title': 'Castles'}, {'a...",139.714234,aPXo4l_JGB_j9v6e6eECvQ,palais-guëll-barcelona,https://s3-media3.fl.yelpcdn.com/bphoto/F1P5Tj...,False,https://www.yelp.com/biz/palais-gu%C3%ABll-bar...,...,2.172368,Carrer De Nou Rambla,,,Barcelona,08001,ES,B,"[Carrer De Nou Rambla, 08001 Barcelona, Spain]",NaN
3,3,None,El Gat de Botero,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",188.343835,U3oCUBTX3ybuyKg9KMwX5Q,el-gat-de-botero-barcelona,https://s3-media2.fl.yelpcdn.com/bphoto/jjyw5T...,False,https://www.yelp.com/biz/el-gat-de-botero-barc...,...,2.169168,"Rambla del Raval, s/n",,None,Barcelona,08001,ES,B,"[Rambla del Raval, s/n, 08001 Barcelona, Spain]",NaN
4,4,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,502,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,503,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,504,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,505,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# cleaning and parsing dataframe for use in analysis
# only distance to bike station and name of venues are important for my anaylsis

yelp_df_cleaning = yelp_df.drop(columns=['fsq_id', 'price'] + list(yelp_df.columns[5:24]))
yelp_df_cleaning2 = yelp_df_cleaning.copy()
yelp_df_cleaning2['venue_type'] = yelp_df_cleaning['categories'].apply(lambda x: [y['title'] for y in x][0] if x is not None else [])

final_yelp_df = yelp_df_cleaning2.drop(columns = 'categories')

# example of resulting dataframe can be found in file: images > yelp_df_examples.png

final_yelp_df

,Bike Stop #,name,distance,location.display_address,venue_type
0,1,None,NaN,NaN,[]
1,2,None,NaN,NaN,[]
2,3,Palais Guëll,139.714234,"[Carrer De Nou Rambla, 08001 Barcelona, Spain]",Castles
3,3,El Gat de Botero,188.343835,"[Rambla del Raval, s/n, 08001 Barcelona, Spain]",Landmarks & Historical Buildings
4,4,None,NaN,NaN,[]
...,...,...,...,...,...
601,502,None,NaN,None,[]
602,503,None,NaN,None,[]
603,504,None,NaN,None,[]
604,505,None,NaN,None,[]


In [19]:
#saved to csv

final_yelp_df.to_csv('yelp.csv')

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Foursquare API provided more complete data compared the the Yelp API. Though the Yelp API had ratings, since my query relates to the proximity of these venues to the usage of bikes in Barcelona, the Yelp API did not provide any benefit. However, the foursquare API returned more results. When searching up with the "query" word of "museums", it returned a larger range of results ranging from art galleries, to concert halls, to monuments and planetariums. Comparatively, Yelp only returned a more limited subset of "museums" including art galleries, landmarks and museums. For future considerations, an expanded budget, and more time, I may be able to query more key words in my API requests to receive a boarder and potentially more full subset of venues that would be relevant to my research question. 

Extra: Since I was unable to do restaurant, here are the top 10 museums sorted by rating in Barcelona from the Yelp API

In [21]:
# read the saved .json file

import pandas as pd
import json

file_path = 'C:/users/ellis/Lighthouse Labs/Lighthouse Tests +Projects/Statistical Modelling with Python/yelp_output.json'

with open(file_path, 'r') as file:
    json_data = json.load(file)

In [22]:
# create dataframe from the .json file

venue_num = 0
ratings = []
names = []
reviews = []

for bike_stop in json_data:
    try: 
        for venue in bike_stop['data']['businesses']:
            venue_num += 1
            rating = venue['rating']
            ratings.append(rating)
            name = venue['name']
            names.append(name)
            review = venue['review_count']
            reviews.append(review)
    except (KeyError, IndexError):
        ratings.append(None)
        names.append(None)
        reviews.append(None)

df = pd.DataFrame({'Name': names, 'Rating': ratings, 'Review_num': reviews})
unique_df = df.drop_duplicates(subset=['Name'], keep='first')
unique_df

,Name,Rating,Review_num
0,Palais Guëll,4.5,2.0
1,El Gat de Botero,5.0,14.0
2,Plaça d'Espanya,4.0,25.0
3,Erotic Museum 69,4.5,5.0
4,Museu de l'Eròtica de Barcelona,2.0,2.0
...,...,...,...
123,Museu del Disseny,4.0,13.0
134,Laie CCCB,4.5,20.0
135,Centre de Cultura Contemporanea,4.5,5.0
137,Plaça del MACBA,3.5,2.0


In [23]:
# strictly highest rating museums in barcelona

top_museums_rating = unique_df.sort_values(by=['Rating', 'Review_num'], ascending=[False, False]).head(10)
top_museums_rating

,Name,Rating,Review_num
16,Hi This Is Barcelona,5.0,18.0
1,El Gat de Botero,5.0,14.0
10,Spanish Trails,5.0,9.0
109,Hash Marihuana Cáñamo & Hemp Museum,5.0,5.0
17,Antiga Sinagoga Major de Barcelona,5.0,4.0
45,Can Framis,5.0,3.0
21,Museo de la Musica,5.0,1.0
6,Basílica de la Sagrada Família,4.5,1136.0
36,Casa Batlló,4.5,348.0
23,La Pedrera - Casa Milà,4.5,263.0


In [24]:
# most reviews - may have more importance compared to highest 5 star ratings

top_museum_reviews = unique_df.sort_values(by=['Review_num', 'Rating'], ascending=[False, False]).head(10)
top_museum_reviews

,Name,Rating,Review_num
6,Basílica de la Sagrada Família,4.5,1136.0
36,Casa Batlló,4.5,348.0
23,La Pedrera - Casa Milà,4.5,263.0
29,Camp Nou,4.5,238.0
116,Font Màgica de Montjuïc,4.5,164.0
43,Palau de la Música Catalana,4.5,132.0
7,Catedral de Barcelona,4.5,120.0
60,Arc de Triomf,4.0,81.0
28,Barri Gòtic,4.5,71.0
54,MACBA - Museu d'Art Contemporani de Barcelona,3.5,55.0


Get the top 10 restaurants according to their rating

Throughout the EDA process, columns were checked to be not null if there was a name available. Each column was also cleaned of values that did not make sense including the deletion of a few bike stops with 0 free bikes but also 0 empty slots. This ensured that my data modelling was as accurate as possible. I also created some new columns that would form the basis of the regression model: number of venues nearby to a bike stop and the percent of bikes being used from a bike stop. These values ensured that I would have a easier time building a single linear regression model that combined all the data I had gathered into a usable and reliable format.



In [ ]:
# I was not expecting to need to do restaurants so I ran out of API requests
# theoretically, this would allow me to sort by ratings though

import requests

yelp_restaurants = []

for bike_stop in bikes_coordinates_dict.values():
    index = bike_stop['index']
    lat, lon = bike_stop['coordinates']

    url = f"https://api.yelp.com/v3/businesses/search"
    
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer ' + open('C:/users/ellis/Lighthouse Labs/Lighthouse Tests +Projects/Statistical Modelling with Python/YelpApiKey.txt').readlines()[0].strip()
    }

    params = {
        'latitude': lat,
        'longitude': lon,
        'term': 'restaurants',
        'radius': 200
    }

    yelp_restaurants = requests.get(url, headers=headers, params=params)
    
    yelp_restaurants.append({'index': index, "data": yelp_response.json()})

yelp_restaurants

In [ ]:
# yelp restaurants into a dataframe

yelp_restaurant_df = pd.DataFrame()

for restaurants in yelp_restaurants:
    data = response['data']
    
    normalized_restaurant_data = pd.json_normalize(data['businesses'])
    yelp_restaurant_df = pd.concat([yelp_restaurant_df, normalized_restaurant_data], ignore_index=True)

yelp_restaurant_df

In [ ]:
# sort dataframe ordered by best rating

yelp_restaurant_df_sorted = yelp_restaurant_df.sort_values(by='rating', ascending=False)
print(yelp_df_sorted)